# PyMuPDF4LLMLoader

This notebook provides a quick overview for getting started with `PyMuPDF4LLM` [document loader](https://python.langchain.com/docs/concepts/document_loaders). For detailed documentation of all __ModuleName__Loader features and configurations head to the [API reference](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyMu4LLMLoader.html).

  

## Overview
### Integration details

| Class                                                                                                                                                  | Package | Local | Serializable | JS support|
|:-------------------------------------------------------------------------------------------------------------------------------------------------------| :--- | :---: | :---: |  :---: |
| [PyMuPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyMu4LLMLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ | 

---------   

### Loader features

| Source | Document Lazy Loading | Native Async Support | Extract Images | Extract Tables |
| :---: | :---: | :---: | :---: |:---: |
| PyMuPDFLoader | ✅ | ❌ | ✅ | ✅ |

  

## Setup

### Credentials

No credentials are required to use PyMuPDF4LLM

If you want to get automated best in-class tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [1]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

Install **langchain_community** and **pymupdf**.

In [2]:
%pip install -qU langchain_community pymupdf4llm
%pip install -qq ../../../../dist/patch_langchain_pdf_loader*.whl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initialization

Now we can instantiate our model object and load documents:

In [3]:
from patch_langchain_community.document_loaders import PyMuPDF4LLMLoader

file_path = "./example_data/layout-parser-paper.pdf"
loader = PyMuPDF4LLMLoader(file_path)

## Load

In [4]:
docs = loader.load()
docs[0]

Document(metadata={'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationdate': '2021-06-22T01:27:10+00:00', 'moddate': '2021-06-22T01:27:10+00:00', 'trapped': '', 'encryption': 'None', 'source': './example_data/layout-parser-paper.pdf', 'file_path': './example_data/layout-parser-paper.pdf', 'total_pages': 16, 'page_count': 16, 'page': 1}, page_content='```\nLayoutParser: A Unified Toolkit for Deep\n\n## Learning Based Document Image Analysis\n\n```\n\nZejiang Shen[1] (�), Ruochen Zhang[2], Melissa Dell[3], Benjamin Charles Germain\nLee[4], Jacob Carlson[3], and Weining Li[5]\n\n1 Allen Institute for AI\n```\n              shannons@allenai.org\n\n```\n2 Brown University\n```\n             ruochen zhang@brown.edu\n\n```\n3 Harvard University\n_{melissadell,jacob carlson}@fas.harvard.edu_\n4 University of Washington\n```\n              bcgl@cs.washington.edu\n\n```\n5 University of Waterloo\

In [5]:
import pprint

pprint.pp(docs[0].metadata)

{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.21',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'moddate': '2021-06-22T01:27:10+00:00',
 'trapped': '',
 'encryption': 'None',
 'source': './example_data/layout-parser-paper.pdf',
 'file_path': './example_data/layout-parser-paper.pdf',
 'total_pages': 16,
 'page_count': 16,
 'page': 1}


## Lazy Load


In [6]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        pages = []
len(pages)

1

In [7]:
print(pages[0].page_content[:100])
pprint.pp(pages[0].metadata)

```
LayoutParser: A Unified Toolkit for Deep

## Learning Based Document Image Analysis

```

Zejian
{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.21',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'moddate': '2021-06-22T01:27:10+00:00',
 'trapped': '',
 'encryption': 'None',
 'source': './example_data/layout-parser-paper.pdf',
 'file_path': './example_data/layout-parser-paper.pdf',
 'total_pages': 16,
 'page_count': 16,
 'page': 1}


The metadata attribute contains at least the following keys:
- source
- page (if in mode *page*)
- total_page
- creationdate
- creator
- producer

Additional metadata are specific to each parser.
These pieces of information can be helpful (to categorize your PDFs for example).

## Splitting mode & custom pages delimiter

When loading the PDF file you can split it in two different ways:
- By page
- As a single text flow

**FIXME** By default PyMuPDF4LLMLoader will split the PDF by page.

### Extract the PDF by page. Each page is extracted as a langchain Document object:

In [8]:
loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

16
{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.21',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'moddate': '2021-06-22T01:27:10+00:00',
 'trapped': '',
 'encryption': 'None',
 'source': './example_data/layout-parser-paper.pdf',
 'file_path': './example_data/layout-parser-paper.pdf',
 'total_pages': 16,
 'page_count': 16,
 'page': 1}


In this mode the pdf is split by pages and the resulting Documents metadata contains the page number. But in some cases we could want to process the pdf as a single text flow (so we don't cut some paragraphs in half). In this case you can use the *single* mode :

### Extract the whole PDF as a single langchain Document object:

In [9]:
loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

1
{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.21',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'moddate': '2021-06-22T01:27:10+00:00',
 'trapped': '',
 'encryption': 'None',
 'source': './example_data/layout-parser-paper.pdf',
 'file_path': './example_data/layout-parser-paper.pdf',
 'total_pages': 16,
 'page_count': 16,
 'page': 1}


Logically, in this mode, the ‘page_number’ metadata disappears. Here's how to clearly identify where pages end in the text flow :

### Add a custom *pages_delimitor* to identify where are ends of pages in *single* mode:

In [10]:
loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
    pages_delimitor="\n-------THIS IS A CUSTOM END OF PAGE-------\n",
)
docs = loader.load()
print(docs[0].page_content[:5780])

```
LayoutParser: A Unified Toolkit for Deep

## Learning Based Document Image Analysis

```

Zejiang Shen[1] (�), Ruochen Zhang[2], Melissa Dell[3], Benjamin Charles Germain
Lee[4], Jacob Carlson[3], and Weining Li[5]

1 Allen Institute for AI
```
              shannons@allenai.org

```
2 Brown University
```
             ruochen zhang@brown.edu

```
3 Harvard University
_{melissadell,jacob carlson}@fas.harvard.edu_
4 University of Washington
```
              bcgl@cs.washington.edu

```
5 University of Waterloo
```
              w422li@uwaterloo.ca

```

**Abstract. Recent advances in document image analysis (DIA) have been**
primarily driven by the application of neural networks. Ideally, research
outcomes could be easily deployed in production and extended for further
investigation. However, various factors like loosely organized codebases
and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been on-going
ef

This could simply be \n, or \f to clearly indicate a page change, or \<!-- PAGE BREAK --> for seamless injection in a Markdown viewer without a visual effect.

# Extract images from the PDF

You can extract images from your PDFs with a choice of three different solutions:
- rapidOCR (lightweight Optical Character Recognition tool)
- Tesseract (OCR tool with high precision)
- Multimodal language model

You can tune these functions to choose the output format of the extracted images among *html*, *markdown* or *text*

The result is inserted between the last and the second-to-last paragraphs of text of the page.

### Extract images from the PDF with rapidOCR:

In [11]:
%pip install -qU rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [12]:
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_text_with_rapidocr,
)

loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_text_with_rapidocr(format="html"),
)
docs = loader.load()

print(docs[5].page_content)

TypeError: to_markdown() got an unexpected keyword argument 'images_to_text'

Be careful, RapidOCR is designed to work with Chinese and English, not other languages.

### Extract images from the PDF with Tesseract:

In [13]:
%pip install -qU pytesseract

Note: you may need to restart the kernel to use updated packages.


In [14]:
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_text_with_tesseract,
)

loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_text_with_tesseract(format="text"),
)
docs = loader.load()
print(docs[5].page_content)

TypeError: to_markdown() got an unexpected keyword argument 'images_to_text'

### Extract images from the PDF with multimodal model:

In [15]:
%pip install -qU langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key =")

In [18]:
from langchain_openai import ChatOpenAI
from patch_langchain_community.document_loaders.parsers.pdf import (
    convert_images_to_description,
)

loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_images=True,
    images_to_text=convert_images_to_description(
        model=ChatOpenAI(model="gpt-4o", max_tokens=1024), format="markdown"
    ),
)
docs = loader.load()
print(docs[5].page_content)

TypeError: to_markdown() got an unexpected keyword argument 'images_to_text'

# Extract tables from the PDF

With PyMUPDF you can extract tables from your PDFs in *html*, *markdown* or *csv* format :

In [19]:
loader = PyMuPDF4LLMLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_tables="markdown",
)
docs = loader.load()
print(docs[4].page_content)

`LayoutParser: A Unified Toolkit for DL-Based DIA` 5

Table 1: Current layout detection models in the LayoutParser model zoo

**Dataset** **Base Model[1]** **Large Model** **Notes**

PubLayNet [38] F / M M Layouts of modern scientific documents
PRImA [3] M    - Layouts of scanned modern magazines and scientific reports
Newspaper [17] F   - Layouts of scanned US newspapers from the 20th century
TableBank [18] F F Table region on modern scientific and business document
HJDataset [31] F / M   - Layouts of history Japanese documents

1 For each dataset, we train several models of different sizes for different needs (the trade-off between accuracy
vs. computational cost). For “base model” and “large model”, we refer to using the ResNet 50 or ResNet 101
backbones [13], respectively. One can train models of different architectures, like Faster R-CNN [28] (F) and Mask
R-CNN [12] (M). For example, an F in the Large Model column indicates it has a Faster R-CNN model trained
using the ResNet 101 

## Working with Files

Many document loaders involve parsing files. The difference between such loaders usually stems from how the file is parsed, rather than how the file is loaded. For example, you can use `open` to read the binary content of either a PDF or a markdown file, but you need different parsing logic to convert that binary data into text.

As a result, it can be helpful to decouple the parsing logic from the loading logic, which makes it easier to re-use a given parser regardless of how the data was loaded.
You can use this strategy to analyze different files, with the same parsing parameters.

In [20]:
from patch_langchain_community.document_loaders.parsers import PyMuPDF4LLMParser
from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./example_data/",
        glob="*.pdf",
    ),
    blob_parser=PyMuPDF4LLMParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

```
LayoutParser: A Unified Toolkit for Deep

## Learning Based Document Image Analysis

```

Zejiang Shen[1] (�), Ruochen Zhang[2], Melissa Dell[3], Benjamin Charles Germain
Lee[4], Jacob Carlson[3], and Weining Li[5]

1 Allen Institute for AI
```
              shannons@allenai.org

```
2 Brown University
```
             ruochen zhang@brown.edu

```
3 Harvard University
_{melissadell,jacob carlson}@fas.harvard.edu_
4 University of Washington
```
              bcgl@cs.washington.edu

```
5 University of Waterloo
```
              w422li@uwaterloo.ca

```

**Abstract. Recent advances in document image analysis (DIA) have been**
primarily driven by the application of neural networks. Ideally, research
outcomes could be easily deployed in production and extended for further
investigation. However, various factors like loosely organized codebases
and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been on-going
ef

It is possible to work with files from cloud storage.

In [21]:
from langchain_community.document_loaders import CloudBlobLoader
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader(
    blob_loader=CloudBlobLoader(
        url="s3:/mybucket", # Supports s3://, az://, gs://, file:// schemes.
        glob="*.pdf",
    ),
    blob_parser=PyMuPDF4LLMParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

ImportError: cannot import name 'CloudBlobLoader' from 'langchain_community.document_loaders' (/home/pprados/workspace.bda/patch_langchain_common/.venv/lib/python3.12/site-packages/langchain_community/document_loaders/__init__.py)

## API reference

For detailed documentation of all `PyMuPDFLoader` features and configurations head to the API reference: https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyMuPDF4LLMPLoader.html